<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Bot con Spacy utilizando un corpus de wikipedia


In [42]:
import bs4 as bs  # Para leer y parsear el texto en HTML de wikipedia
import gradio as gr
import json
import lxml
import nltk
import numpy as np
import random
import re
import spacy
import ssl
import string
import urllib.request
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from spacy.lang.es.stop_words import STOP_WORDS as es_stop

In [24]:
context = ssl._create_unverified_context()

In [25]:
nlp = spacy.load('es_core_news_sm')


### Datos
Se consumirán los datos del artículo de wikipedia sobre el deporte "Tennis" en inglés.

In [47]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/Tres_leyes_de_la_rob%C3%B3tica', context=context)
raw_html = raw_html.read()

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [48]:
# Demos un vistazo
article_text

'las tres leyes de la robótica son un conjunto de normas elaboradas por el escritor de ciencia ficción isaac asimov que se aplican a la mayoría de los robots de sus obras y que están diseñados para cumplir órdenes. aparecidas por primera vez en el relato círculo vicioso (runaround) de 1942, establecen lo siguiente:\nestas leyes forman un principio organizador y un tema unificador para la ficción basada en robótica de asimov, que aparece en su serie robot, las historias vinculadas a ella y su serie lucky starr de ficción para adultos jóvenes. en ese universo, las leyes son «formulaciones matemáticas impresas en los senderos positrónicos del cerebro» de los robots (líneas de código del programa que regula el cumplimiento de las leyes guardado en la memoria principal de éstos), y no se pueden eludir, ya que están pensadas como una característica de seguridad.\nlas leyes originales han sido modificadas y desarrolladas por asimov y otros autores. el propio asimov hizo ligeras modificaciones

In [49]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 15057


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [50]:
# el inicio con 'r' antes de cada string indica que se interprete como raw string
# '\n' es interpretado por Python como salto de linea
# r'\n' es interpretado por Python como el string formado por dos caracteres: 
#  backslash y n

# substituir con regex con espacio vacío:
text = re.sub(r'\[[0-9]*\]', ' ', article_text) # substituir los números entre corchetes

text = re.sub(r'\s+', ' ', text) # substituir más de un caracter de espacio, salto de línea o tabulación

text = re.sub(r'\u200b', '', text) # elimino zero width space

text = re.sub(r'  ', ' ', text)

In [51]:
# Demos un vistazo
text

'las tres leyes de la robótica son un conjunto de normas elaboradas por el escritor de ciencia ficción isaac asimov que se aplican a la mayoría de los robots de sus obras y que están diseñados para cumplir órdenes. aparecidas por primera vez en el relato círculo vicioso (runaround) de 1942, establecen lo siguiente: estas leyes forman un principio organizador y un tema unificador para la ficción basada en robótica de asimov, que aparece en su serie robot, las historias vinculadas a ella y su serie lucky starr de ficción para adultos jóvenes. en ese universo, las leyes son «formulaciones matemáticas impresas en los senderos positrónicos del cerebro» de los robots (líneas de código del programa que regula el cumplimiento de las leyes guardado en la memoria principal de éstos), y no se pueden eludir, ya que están pensadas como una característica de seguridad. las leyes originales han sido modificadas y desarrolladas por asimov y otros autores. el propio asimov hizo ligeras modificaciones a

In [52]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 14944


### 3 - Tokenización

In [53]:
doc = nlp(text)
lexical_tokens = [t.orth_ for t in doc if not t.is_punct | t.is_stop]
lexical_tokens

['leyes',
 'robótica',
 'conjunto',
 'normas',
 'elaboradas',
 'escritor',
 'ciencia',
 'ficción',
 'isaac',
 'asimov',
 'aplican',
 'mayoría',
 'robots',
 'obras',
 'diseñados',
 'cumplir',
 'órdenes',
 'aparecidas',
 'relato',
 'círculo',
 'vicioso',
 'runaround',
 '1942',
 'establecen',
 'leyes',
 'forman',
 'principio',
 'organizador',
 'tema',
 'unificador',
 'ficción',
 'basada',
 'robótica',
 'asimov',
 'aparece',
 'serie',
 'robot',
 'historias',
 'vinculadas',
 'serie',
 'lucky',
 'starr',
 'ficción',
 'adultos',
 'jóvenes',
 'universo',
 'leyes',
 'formulaciones',
 'matemáticas',
 'impresas',
 'senderos',
 'positrónicos',
 'cerebro',
 'robots',
 'líneas',
 'código',
 'programa',
 'regula',
 'cumplimiento',
 'leyes',
 'guardado',
 'memoria',
 'principal',
 'eludir',
 'pensadas',
 'característica',
 'seguridad',
 'leyes',
 'originales',
 'modificadas',
 'desarrolladas',
 'asimov',
 'autores',
 'asimov',
 'ligeras',
 'modificaciones',
 'libros',
 'historias',
 'cortas',
 'desarr

In [54]:
corpus = []
for sent_id, sent in enumerate(doc.sents):
    corpus.append(str(sent))

### 4 - Lematización, Remoción de Simbolos de puntuación
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [55]:
# Tokenization & lemmatization
lemma_list = []
for token in doc:
    lemma_list.append(token.lemma_)
print(lemma_list)

['el', 'tres', 'ley', 'de', 'el', 'robótica', 'ser', 'uno', 'conjunto', 'de', 'norma', 'elaborado', 'por', 'el', 'escritor', 'de', 'ciencia', 'ficción', 'isaac', 'asimov', 'que', 'él', 'aplicar', 'a', 'el', 'mayoría', 'de', 'el', 'robot', 'de', 'su', 'obra', 'y', 'que', 'estar', 'diseñado', 'para', 'cumplir', 'órden', '.', 'aparecido', 'por', 'primero', 'vez', 'en', 'el', 'relato', 'círculo', 'vicioso', '(', 'runaround', ')', 'de', '1942', ',', 'establecer', 'él', 'siguiente', ':', 'este', 'ley', 'formar', 'uno', 'principio', 'organizador', 'y', 'uno', 'tema', 'unificador', 'para', 'el', 'ficción', 'basado', 'en', 'robótica', 'de', 'asimov', ',', 'que', 'aparecer', 'en', 'su', 'serie', 'robot', ',', 'el', 'historia', 'vinculado', 'a', 'él', 'y', 'su', 'serie', 'lucky', 'starr', 'de', 'ficción', 'para', 'adulto', 'joven', '.', 'en', 'ese', 'universo', ',', 'el', 'ley', 'ser', '«', 'formulación', 'matemático', 'impreso', 'en', 'el', 'sendero', 'positrónico', 'del', 'cerebro', '»', 'de', 

In [56]:
# Stop words
filtered_sentence =[]
for word in lemma_list:
    # word es un string, para recuperar la información de los objetos de SpaCy
    # necesitamos usar el string para pasar a un lexema, el objeto de SpaCy
    # que para cada término contiene la información del preprocesamiento
    # (se podría también directamente filtrar stopwords en el paso de lematización)
    lexeme = nlp.vocab[word]
    if lexeme.is_stop == False:
        filtered_sentence.append(word)
filtered_sentence 

['ley',
 'robótica',
 'conjunto',
 'norma',
 'elaborado',
 'escritor',
 'ciencia',
 'ficción',
 'isaac',
 'asimov',
 'aplicar',
 'mayoría',
 'robot',
 'obra',
 'diseñado',
 'cumplir',
 'órden',
 '.',
 'aparecido',
 'relato',
 'círculo',
 'vicioso',
 '(',
 'runaround',
 ')',
 '1942',
 ',',
 'establecer',
 ':',
 'ley',
 'formar',
 'principio',
 'organizador',
 'tema',
 'unificador',
 'ficción',
 'basado',
 'robótica',
 'asimov',
 ',',
 'aparecer',
 'serie',
 'robot',
 ',',
 'historia',
 'vinculado',
 'serie',
 'lucky',
 'starr',
 'ficción',
 'adulto',
 'joven',
 '.',
 'universo',
 ',',
 'ley',
 '«',
 'formulación',
 'matemático',
 'impreso',
 'sendero',
 'positrónico',
 'cerebro',
 '»',
 'robot',
 '(',
 'línea',
 'código',
 'programa',
 'regular',
 'cumplimiento',
 'ley',
 'guardado',
 'memoria',
 'principal',
 ')',
 ',',
 'eludir',
 ',',
 'pensado',
 'característica',
 'seguridad',
 '.',
 'ley',
 'original',
 'modificar',
 'desarrollado',
 'asimov',
 'autor',
 '.',
 'asimov',
 'ligero',

In [57]:
# Filter punctuation
filtered_sentence = [w for w in filtered_sentence if w not in string.punctuation+'—']
filtered_sentence

['ley',
 'robótica',
 'conjunto',
 'norma',
 'elaborado',
 'escritor',
 'ciencia',
 'ficción',
 'isaac',
 'asimov',
 'aplicar',
 'mayoría',
 'robot',
 'obra',
 'diseñado',
 'cumplir',
 'órden',
 'aparecido',
 'relato',
 'círculo',
 'vicioso',
 'runaround',
 '1942',
 'establecer',
 'ley',
 'formar',
 'principio',
 'organizador',
 'tema',
 'unificador',
 'ficción',
 'basado',
 'robótica',
 'asimov',
 'aparecer',
 'serie',
 'robot',
 'historia',
 'vinculado',
 'serie',
 'lucky',
 'starr',
 'ficción',
 'adulto',
 'joven',
 'universo',
 'ley',
 '«',
 'formulación',
 'matemático',
 'impreso',
 'sendero',
 'positrónico',
 'cerebro',
 '»',
 'robot',
 'línea',
 'código',
 'programa',
 'regular',
 'cumplimiento',
 'ley',
 'guardado',
 'memoria',
 'principal',
 'eludir',
 'pensado',
 'característica',
 'seguridad',
 'ley',
 'original',
 'modificar',
 'desarrollado',
 'asimov',
 'autor',
 'asimov',
 'ligero',
 'modificación',
 'libro',
 'historia',
 'corto',
 'desarrollar',
 'robot',
 'interactuae

### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus del artículo de wikipedia

In [39]:
def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del ingles y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(stop_words=list(es_stop))

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada.

In [58]:
def bot_response(human_text):
    print("Q:", human_text)    
    resp = generate_response(human_text.lower(), corpus)
    print("A:", resp)
    return resp

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `layout` parameter is deprecated, and it has no effect
  warnings.warn(value)


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Q: ley
A: , lo que equivale a la segunda ley de la robótica.
Q: robótica
A: , lo que equivale a la segunda ley de la robótica.
Q: leyes
A: las leyes se aplican, por supuesto, a todas las herramientas que utilizan los seres humanos", y "los análogos de las leyes están implícitos en el diseño de casi todas las herramientas, robóticas o no": asimov creía que, idealmente, los humanos también seguirían las leyes: mi respuesta es: "sí, las tres leyes son la única forma en que los seres humanos racionales pueden lidiar con los robots, o con cualquier otra cosa".
Q: cero
A: en la ficción posterior donde los robots habían asumido la responsabilidad del gobierno de planetas enteros y civilizaciones humanas, asimov también agregó una cuarta, o ley cero, para preceder a las demás: las tres leyes y la ley cero han impregnado la ciencia ficción y se mencionan en muchos libros, películas y otros medios.
Q: primera
A: I am sorry, I could not understand you
Q: primer
A: I am sorry, I could not understa

(<gradio.routes.App at 0x1389d6230>, 'http://127.0.0.1:7860/', None)

### 7 -  Conclusiones

- Tomar un ejemplo de los bots utilizados (uno de los dos) y construir el propio.
- Sacar conclusiones de los resultados.

__IMPORTANTE__: Recuerde para la entrega del ejercicio debe quedar registrado en el colab las preguntas y las respuestas del BOT para que podamos evaluar el desempeño final.

##### Registro Completo de Preguntas y Respuestas

Q: ley  
A: , lo que equivale a la segunda ley de la robótica.  
Q: robótica  
A: , lo que equivale a la segunda ley de la robótica.  
Q: leyes  
A: las leyes se aplican, por supuesto, a todas las herramientas que utilizan los seres humanos", y "los análogos de las leyes están implícitos en el diseño de casi todas las herramientas, robóticas o no": asimov creía que, idealmente, los humanos también seguirían las leyes: mi respuesta es: "sí, las tres leyes son la única forma en que los seres humanos racionales pueden lidiar con los robots, o con cualquier otra cosa".  
Q: cero  
A: en la ficción posterior donde los robots habían asumido la responsabilidad del gobierno de planetas enteros y civilizaciones humanas, asimov también agregó una cuarta, o ley cero, para preceder a las demás: las tres leyes y la ley cero han impregnado la ciencia ficción y se mencionan en muchos libros, películas y otros medios.  
Q: primera  
A: I am sorry, I could not understand you  
Q: primer  
A: I am sorry, I could not understand you  
Q: segunda  
A: I am sorry, I could not understand you  
Q: tercera  
A: I am sorry, I could not understand you  
Q: moral  
A: las tres leyes de la robótica representan el código moral del robot.  
Q: Ley 1  
A: , lo que equivale a la segunda ley de la robótica.  
Q: daño  
A: en contra de causar daño a los humanos).  
Q: dañar  
A: calvin señala que típicamente se espera que los seres humanos se abstengan de dañar a otros seres humanos (excepto en tiempos de extrema coacción como la guerra, o para salvar a un número mayor) y esto es equivalente a la primera ley de un robot.  
Q: protección  
A: estas tres leyes surgen únicamente como medida de protección para los seres humanos.  
Q: asimov  
A: asimov admiró la historia.  
Q: matemática  
A: I am sorry, I could not understand you  
Q: matemáticas  
A: en ese universo, las leyes son «formulaciones matemáticas impresas en los senderos positrónicos del cerebro» de los robots (líneas de código del programa que regula el cumplimiento de las leyes guardado en la memoria principal de éstos), y no se pueden eludir, ya que están pensadas como una característica de seguridad.  
Q: robots  
A: los robots son esencialmente decentes".  
Q: robot  
A: ¿es posible que un robot "dañe" a un ser humano?  
Q: órdenes  
A: las tres leyes de la robótica son un conjunto de normas elaboradas por el escritor de ciencia ficción isaac asimov que se aplican a la mayoría de los robots de sus obras y que están diseñados para cumplir órdenes.  